# Improvements to Astropy Modelling for SPICE Data Fitting

This set of notebooks gives an overview of the improvements that have been made to Astropy aimed at speeding up spectral fitting of SPICE data.

In the process of this work we have submitted (and have had accepted) the following contributions to Astropy, these will be released in the upcoming astropy 7.0 release expected in November.

#### Significant Changes

* ["Improve fitting performance of most non-linear fitters"](https://github.com/astropy/astropy/pull/16673) - This patch worked on speeding up the objective function used in the fitting loop. This function is called a very large number of times during fitting (both for evaluation of the fit and also computation of the analytical derivitive) so the performance gains here had a large impact. Most of the work here focused on reducing the overhead in validating the inputs, which isn't needed in the fitter.
* ["Add function to carry out parallel fitting on N-dimensional datasets"](https://github.com/astropy/astropy/pull/16696) - This added the new `parallel_fit_dask` function which uses the [dask](https://www.dask.org/) library to parallelise the fitting of any model with any non-linear fitter.
* ["Provide a way to reinitialize the parameter values of a model"](https://github.com/astropy/astropy/pull/16812) - This patch was required to allow us to convert a model with scalar parameters to a model with array parameters, this was required to allow the user to specify a model for a single spectra and have that model be returned as a model for each spectra in the raster.
* ["Add has_tied, has_fixed and has_bounds properties to Model"](https://github.com/astropy/astropy/pull/16677) - This patch also speed up performance of fitting by caching some of the properties of models which decide what paths the code takes inside the objective function.
* ["Speed up model evaluation for some use cases"](https://github.com/astropy/astropy/pull/16670) - This work improved the performance of computing a model, by speeding up some of the validation performed. This PR ended up not affecting the fitting performance as we bypassed the whole code path completely in other changes. This still has a noticable impact when evaluating the large grid of fitted models however.

#### Supporting Changes

* ["Fix typo in `_NonLinearLSQFitter.__call__`"](https://github.com/astropy/astropy/pull/16664)
* ["Allow Unit.to to take and return dask arrays"](https://github.com/astropy/astropy/pull/16613)
* ["Fix removal of units from CompoundModel with * or / and fix fitting of these models"](https://github.com/astropy/astropy/pull/16678)
* ["Reduce duplication of CompoundModel._initialize_slices"](https://github.com/astropy/astropy/pull/16709)
* ["Simplify construction of new CompoundModel in with_units_for_data"](https://github.com/astropy/astropy/pull/16712)

In the following notebooks we show code examples of both the pyCFIT workflow and the new `parallel_fit_dask` workflow.

### pyCFIT

In the [pyCFIT notebook](./CFIT%20Example.ipynb) we compare fitting each spectra in a 160x160 grid before and after the performance changes in Astropy. In the most recent release of Astropy (version 6.1.2), before any of our changes, fitting this grid took **3 minutes 51 seconds** on a 16 core workstation. With our performance improvements to Astropy it took **1 minute 52 seconds** or 2x faster.

### New Dask Workflow

In the [New Astropy Demo](./New%20Astropy%20Demo.ipynb) notebook we demonstrate fitting the same model to the same data as in the pyCFIT notebook. This new function fitted the raster in **31s or nearly 7.5x faster** than the original pyCFIT example. Dask allows scaling to very large distributed compute, such as HPC or cloud resources, so this can be made faster with more compute power.

## Comparisions between pyCFIT and New Astropy

We believe that the new astropy function encapsulates all the existing non-interactive functionality in pyCFIT, but executes the fitting in parallel. We have also tested the new Astropy functionality with various other sources of data, such as VISP, EIS and Astronomical data, including using cloud compute to perform the fitting via [Dask Distributed](https://distributed.dask.org/).

For more examples of using the new astropy functionality see this repo: https://github.com/aperiosoftware/parallel-modeling-examples/

## Next Steps

We believe there are still further improvements to the performance of fitting in Astropy which could be made in the remaining ~1/4 of the project. Our testing indicates the parallism overhead is still minimal in comparision to the fitting time of a single spectra, so this is where we would focus our efforts. One area we still haven't addressed is the analytical computation of the derivitives. We believe this is responsible currently for approximately 38% of the fitting time. As the computation of the numerical derivitive requires two evaluations of the objective function it could be possible to reduce the time computing the derivities by approximately 50% by implementing an analytical computation of the derivitive.

In addition to this there are a few more minor things we would like to address such as the astropy issue ["Make fitters more robust against differences in scale"](https://github.com/astropy/astropy/issues/6269) as this prevents using the WCS object (where the wavelength is in meters) with a model in Angstroms directly. Finally, again to increase usability we would like to add support for passing a `NDData` (or `NDCube`) object to `parallel_fit_dask` to simplify the API.